# 기초통계 및 검정통계 마무리 문제

### 데이터 로딩 (20점)

In [1]:
import pandas as pd
from scipy import stats
from statsmodels.stats import multicomp

In [2]:
df = pd.read_csv("datas/cdn_page_load_times.csv")
df.head()

,CDN_A,CDN_B,CDN_C
0,103.115543,102.342433,98.371838
1,94.341060,118.835131,86.193353
2,114.586739,104.359728,79.047870
3,156.155017,90.357875,63.730201
4,156.920524,75.013771,116.655484


In [3]:
A = df['CDN_A'].dropna().values
B = df['CDN_B'].dropna().values
C = df['CDN_C'].dropna().values

### 가설검정 방법 (일원분산분석 채택) (20점)

 - 귀무가설 : 세 CDN의 평균 로드 시간은 모두 같다.
 - 대립가설 : 적어도 하나의 CDN은 평균 로드 시간이 다르다.

### 정규성 검정 (20점)

In [4]:
print(stats.shapiro(A-B))
print(stats.shapiro(A-C))
print(stats.shapiro(B-C))

ShapiroResult(statistic=np.float64(0.9940977465531138), pvalue=np.float64(0.9446069376138456))
ShapiroResult(statistic=np.float64(0.9898981527862001), pvalue=np.float64(0.6563609451801026))
ShapiroResult(statistic=np.float64(0.9821121731681786), pvalue=np.float64(0.19351085781526473))


 - 모두 p-value > 0.05 : 각 변수간의 차이가, 모두 정규 분포를 뛴다는 것을 알 수 있음.

### 등분산성 검정

In [5]:
levene_stat, levene_p_value = stats.levene(A, B, C, center='median')
print(f"levene_stat: {levene_stat}, levene_p_value: {levene_p_value}")

levene_stat: 0.6331383822746123, levene_p_value: 0.5316379878247817


 - p-value > 0.05, 샘플 간 분산이 유사하다고 판단할 수 있음

### 가설검정 수행 (20점)

In [6]:
print(stats.f_oneway(A, B, C))

F_onewayResult(statistic=np.float64(39.192607237897704), pvalue=np.float64(7.842852500131719e-16))


 - p-value < 0.05 : 거의 0에 가까우므로, 귀무가설을 강력하게 기각함 (e-16)
 - 최소 하나의 그룹은 다른 그룹의 평균과 유의미하게 다르다고 판단할 수 있음

### 사후성 검정 (20점)

In [7]:
Data = A.tolist() + B.tolist() + C.tolist()
# 각 값이 속한 그룹 이름
Group = ['A'] * len(A) + ['B'] * len(B) + ['C'] * len(C)

print(multicomp.pairwise_tukeyhsd(Data, Group))

 Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower    upper  reject
-----------------------------------------------------
     A      B -24.8154    0.0 -31.4178 -18.213   True
     A      C -12.5563    0.0 -19.1587 -5.9538   True
     B      C  12.2591 0.0001   5.6567 18.8616   True
-----------------------------------------------------


 - reject가 모두 True이고 p-adj < 0.05이므로 모두 유의미하게 다르다는 의미
 - meandiff는 group2 평균 - group1 평균이므로 음수면 group2의 크기가 더 작다는 의미
 - A가 B보다 약 24만큼 크고 C보다 약 12만큼 크며 B가 C보다 12만큼 작음
 - 로드 시간이 작을수록 빠르므로 가장 빠른 CDN은 B
 - B를 채택해야 함